In [ ]:
#installs e libs
!pip install pysendpulse
!pip install IPython
!pip install pandas
from pysendpulse.pysendpulse import PySendPulse
from IPython.display import display
from pandas import json_normalize
import collections
import datetime
import requests
import base64
import json
import pandas as pd

  Preparing metadata (setup.py) ... done
  Created wheel for python3-memcached: filename=python3_memcached-1.51-py3-none-any.whl size=14382 sha256=273ad9be3538435ce3d988e6b360e1ddbabfdfb09435f95df8a408202adba455
  Stored in directory: /root/.cache/pip/wheels/50/02/82/7d651cbabbea97f8214de366974ca37cf7dc8c490d6d2235dd
Successfully built python3-memcached
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.8 MB/s eta 0:00:00


In [ ]:
url = "https://api.sendpulse.com/oauth/access_token"

payload = {
    "grant_type": "client_credentials",
    "client_id": "",
    "client_secret": ""
}

response = requests.post(url, json=payload)


access_token = json.loads(response.text)['access_token']


headers = {
    'Authorization': f'Bearer {access_token}'
}

In [ ]:
sms_campanhas = requests.get('https://api.sendpulse.com/sms/campaigns/list', headers=headers)
sms_campanhas.json()
print(sms_campanhas.json())

In [ ]:
df_sms = pd.DataFrame(sms_campanhas.json())
df_sms.to_csv('df_sms.csv', index=False)

In [ ]:
df_sms_expandido = json_normalize(df_sms['data']) # Expande a coluna 'data'

df_sms_desaninhado = pd.concat([df_sms, df_sms_expandido], axis=1) # Concatena os DataFrames

df_sms_desaninhado = df_sms_desaninhado.drop(['data'], axis=1)

print(df_sms_desaninhado)

In [ ]:
df_sms_desaninhado.to_csv('df_sms_desaninhado.csv', index=False)

In [ ]:
ids = df_sms_desaninhado['id'].tolist()

# Cria uma lista para armazenar os dados de cada campanha
campaign_data = []

# Loop através de cada ID
for id in ids:
    # Constrói a URL com o ID atual
    url = f'https://api.sendpulse.com/sms/campaigns/info/{id}'

    # Faz a requisição GET
    response = requests.get(url, headers=headers)

    # Verifica se a requisição foi bem-sucedida
    if response.status_code == 200:
        # Adiciona os dados da campanha à lista
        campaign_data.append(response.json())

# Cria um novo DataFrame com os dados das campanhas
df_campaigns = pd.DataFrame(campaign_data)

In [ ]:
print(campaign_data)

In [ ]:
df_campaigns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228 entries, 0 to 227
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   result  228 non-null    bool  
 1   data    228 non-null    object
dtypes: bool(1), object(1)
memory usage: 2.1+ KB


In [ ]:
df_info_campanhas = json_normalize(df_campaigns['data'])

df_info_campanhas = pd.concat([df_campaigns, df_info_campanhas], axis=1)

df_info_campanhas = df_info_campanhas.drop(['data'], axis=1)

In [ ]:
df_info_campanhas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228 entries, 0 to 227
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   result            228 non-null    bool  
 1   id                228 non-null    int64 
 2   address_book_id   228 non-null    int64 
 3   currency          228 non-null    object
 4   company_price     228 non-null    object
 5   send_date         228 non-null    object
 6   date_created      228 non-null    object
 7   sender_name       228 non-null    object
 8   task_phones_info  228 non-null    object
dtypes: bool(1), int64(2), object(6)
memory usage: 14.6+ KB


In [ ]:
df_info_campanhas['task_phones_info_count'] = df_info_campanhas['task_phones_info'].apply(len)

In [ ]:
df_info_campanhas.info()

In [ ]:
display(df_info_campanhas)

In [ ]:
def count_explains(events):
  explain_counts = collections.Counter()
  for event in events:
    explain = event.get('status_explain')
    if explain:
      explain_counts[explain] += 1
  return explain_counts

df_info_campanhas['explain_counts'] = df_info_campanhas['task_phones_info'].apply(count_explains)

for explain in df_info_campanhas['explain_counts'].apply(lambda x: x.keys()).explode().unique():
  df_info_campanhas[f'count_{explain}'] = df_info_campanhas['explain_counts'].apply(lambda x: x.get(explain, 0))

# Remover a coluna 'explain_counts' (opcional)
df_info_campanhas_contagem = df_info_campanhas.drop('explain_counts', axis=1)

display(df_info_campanhas_contagem)

In [ ]:
df_info_campanhas_contagem.to_csv('df_info_campanhas_contagem.csv', index=False)

In [ ]:
df_info_campanhas_contagem.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228 entries, 0 to 227
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   result                  228 non-null    bool  
 1   id                      228 non-null    int64 
 2   address_book_id         228 non-null    int64 
 3   currency                228 non-null    object
 4   company_price           228 non-null    object
 5   send_date               228 non-null    object
 6   date_created            228 non-null    object
 7   sender_name             228 non-null    object
 8   task_phones_info        228 non-null    object
 9   task_phones_info_count  228 non-null    int64 
 10  count_Delivered         228 non-null    int64 
 11  count_Not delivered     228 non-null    int64 
 12  count_Ready to send     228 non-null    int64 
dtypes: bool(1), int64(6), object(6)
memory usage: 21.7+ KB


In [ ]:
# Renomear colunas em df_info_campanhas_contagem
df_info_campanhas_contagem = df_info_campanhas_contagem.rename(columns={
    'id' : 'id_campanhas',
    'send_date' : 'data_de_envio',
    'sender_name' : 'objetivo_campanha',
    'task_phones_info_count' : 'sms_enviados',
    'count_Delivered' : 'sms_entregue',
    'count_Not delivered' : 'sms_nao_entregue',
    'count_Ready to send' : 'sms_envios_pendentes'
})

display(df_info_campanhas_contagem)

In [ ]:
df_info_campanhas_contagem.info()

In [ ]:
df_campanhas_sms = df_info_campanhas_contagem[['id_campanhas','data_de_envio','objetivo_campanha','sms_enviados','sms_entregue','sms_nao_entregue','sms_envios_pendentes']]

In [ ]:
df_campanhas_sms.info()

In [ ]:
df_info_campanhas.head()

In [ ]:
df_info_campanhas.to_csv('df_info_campanhas.csv', index=False)

In [ ]:
df_campanhas_sms.to_csv('df_campanhas_sms.csv', index=False)